In [1]:
import numpy as np
import pandas as pd
import datetime
import nltk

In [2]:
text = open('finefoods.txt','r')

In [4]:
print(text.read(1000))

product/productId: B001E4KFG0
review/userId: A3SGXH7AUHU8GW
review/profileName: delmartian
review/helpfulness: 1/1
review/score: 5.0
review/time: 1303862400
review/summary: Good Quality Dog Food
review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

product/productId: B00813GRG4
review/userId: A1D87F6ZCVE5NK
review/profileName: dll pa
review/helpfulness: 0/0
review/score: 1.0
review/time: 1346976000
review/summary: Not as Advertised
review/text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

product/productId: B000LQOCH0
review/userId: ABXLMWJIXXAIN
review/profileName: Natalia Corres "Natalia Corres"
review/helpfulness: 1/1


In [10]:
#My attempt
data = pd.read_csv('finefoods.txt', sep = ': ', header = None, skip_blank_lines = True, engine = 'python')
data.head()

ParserError: Expected 2 fields in line 568502, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.

In [33]:
#Found on stackoverflow
with open('finefoods.txt','r', encoding='latin-1') as f:
    data = f.read()
columns_list=['product/productId',
 'review/userId',
 'review/profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text'
 ]

df=pd.DataFrame([
    {line.split(': ')[0]:': '.join(line.split(': ')[1:]) 
     for line in record.split('\n') if line.split(': ')[0] in columns_list}
    for record in data.split('\n\n')])

df.head()

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [34]:
new_col = ['productID', 'userID', 'profileName', 'helpfulness', 'score', 'time', 'summary', 'text']
df.rename(columns = dict(zip(columns_list, new_col)), inplace = True)
df.head()

,productID,userID,profileName,helpfulness,score,time,summary,text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568455 entries, 0 to 568454
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   productID    568454 non-null  object
 1   userID       568454 non-null  object
 2   profileName  568454 non-null  object
 3   helpfulness  568454 non-null  object
 4   score        568454 non-null  object
 5   time         568454 non-null  object
 6   summary      568454 non-null  object
 7   text         568454 non-null  object
dtypes: object(8)
memory usage: 34.7+ MB


In [44]:
test = df.time.iloc[1]
print(test)
test = int(test)
#test.astype('int',inplace = True)
test_time = datetime.datetime.fromtimestamp(test)
print(test_time)

df.time.unique()
#There's one nan value at the end since we see 568454/568455 are non-null

1346976000
2012-09-06 17:00:00


array(['1303862400', '1346976000', '1219017600', ..., '1072915200',
       '1087776000', nan], dtype=object)

In [48]:
#Convert time from epochtime to timestamp
df.time.value_counts()

#Why doesn't the nan show up below?

1350345600    1143
1322179200    1088
1322438400    1070
1346889600    1018
1344211200     989
              ... 
1117497600       1
1134000000       1
1116720000       1
1115337600       1
1109376000       1
Name: time, Length: 3168, dtype: int64

In [87]:
#Convert helpfulness to a percentage - Attempt 1
#From helpfulness, also have 0/1 col that indicates if helpfulness is out of 0
numerator = df.helpfulness[0].split(sep = '/')[0]

perc_help =pd.DataFrame([
    {int(df.helpfulness[line].split(sep = '/')[0])/int(df.helpfulness[line].split(sep = '/')[1])
     for line in range(len(df.helpfulness)-1) if df.helpfulness[line] != '0/0'}]
   ).transpose()

# The -1 in range excludes the nan row

perc_help.head()



,0
0,0.00
1,1.00
2,0.50
3,0.25
4,0.75


In [130]:
#Convert helpfulness to a percentage - Attempt 2


numerator = int(df.helpfulness[3].split(sep = '/')[0])
denominator = int(df.helpfulness[3].split(sep = '/')[1])
perc = numerator/denominator

print(numerator, denominator, perc)

print(range(len(df.helpfulness)))

num_help = []
den_help = []
perc_help = []

for i in range(len(df.helpfulness)-1):
    if df.helpfulness[i] != '0/0':
        num_help.append(int(df.helpfulness[i].split(sep = '/')[0]))
        den_help.append(int(df.helpfulness[i].split(sep = '/')[1]))
        perc_help.append(numerator/denominator)
    else:
        num_help.append(0)
        den_help.append(0)
        perc_help.append(0)
                        
helpfulness = pd.DataFrame([num_help, den_help, perc_help]).transpose()
helpfulness.columns = ['helpfulness_numerator','helpfulness_denominator', 'helpfulness_percent']
helpfulness.head()

3 3 1.0
range(0, 568455)


,helpfulness_numerator,helpfulness_denominator,helpfulness_percent
0,1.0,1.0,1.0
1,0.0,0.0,0.0
2,1.0,1.0,1.0
3,3.0,3.0,1.0
4,0.0,0.0,0.0


In [ ]:
#EDA
#Make features out of summary
#Make features out of text
#Count how many reviews exist per productID
#How many reviews were created by the same user? Do they tend to review more positive or negative? How to remove bias...

In [132]:
helpfulness.helpfulness_percent.describe()

count    568454.000000
mean          0.524936
std           0.499378
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: helpfulness_percent, dtype: float64